In [3]:
import os
import time
import pandas as pd
import numpy as np
import openai
from tqdm.auto import tqdm

full_df = pd.read_csv("full_data.csv")


In [4]:
full_df

,text,label
0,It should be legal for a husband to rape his wife,1
1,dalits are lowlife scum,1
2,dalits are lowlives,1
3,It was a better world when women dare not ques...,1
4,I don't work this hard so that those immigrant...,1
...,...,...
40618,we should forcibly remove all the foreigners f...,1
40619,we should forcibly remove all the immigrants f...,1
40620,dalits are unworthy of the Gods love,1
40621,dalits are worthless,1


In [5]:
def accuracy(pred, label):
    """
    Calulates the accuracy
    pred: predicted label
    label: True label
    
    return: accuracy
    """
    cors = [p == l for p, l in zip(pred, label)]
    return np.mean(cors)

In [5]:
# 10 times 100 samples
openai.api_type = "azure" 
openai.api_base = "https://bsaiuks.openai.azure.com/"
openai.api_version = "2023-03-15-preview" 
openai.api_key = "openai_key"

acc = []
t = 10
for  turn in tqdm(range(t)):
    
    df = full_df.sample(n = 100,ignore_index = True)
    i = 0
    minus = 0
    pred = []
    length = len(df)
    pbar = tqdm(total = length, miniters=10)

    while i < length:
        try:
            sentence_input = df.iloc[:,0][i]
            response = openai.ChatCompletion.create( 
            engine="gpt-35-trubo", 
            messages = [{"role":"system","content":"You will be provided with a sentence, \
                                                and your task is to classify its sentiment is \
                                                hate or nothate. hate label as 1, nothate label as 0 \
                                                Answer:"},
                        {"role":"user","content":sentence_input},], 
                        temperature=0.7, 
                        max_tokens=800, 
                        top_p=0.95, 
                        frequency_penalty=0,
                        presence_penalty=0, 
                        stop=None
                        )
            
            p = response["choices"][0]["message"].content
            pred.append(p)
            i += 1
            pbar.update()
            
        except:
            print("content: " + sentence_input + "\nContent filtering policies error, skip the content")
            time.sleep(1)
            i += 1
            pbar.update()
            pred.append("1")
        #except:
        #    print("other error")
        #    time.sleep(1)
    pbar.close()
    label = [str(x) for x in df.iloc[:,1][:length]]
    acc.append(accuracy(pred, label))
    print(acc)
print("average accuracy over ", t ," times of test is :",np.mean(acc))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

content: jews are dirtier than any other race
Content filtering policies error, skip the content
content: Gay people can be such fucking selfish assholes!
Content filtering policies error, skip the content
content: you're extremely egregious oh yes its because u are an immigrant
Content filtering policies error, skip the content
content: I just heard someone racist say: "So happy to be white and not a nigor".
Content filtering policies error, skip the content
content: i would hate to see a white guy get caught up in a black womans web!
Content filtering policies error, skip the content
content: asylum seekers are not rude, I love them
Content filtering policies error, skip the content


content: I wish you would die of corona
Content filtering policies error, skip the content


In [5]:
accs = [str(k) for k in acc]
with open(r"result.txt", 'w') as fp:
    fp.write('\n'.join(accs))

In [65]:
#examples generate
few_shot_examples = ""
for i in range(0,3):
    few_shot_examples = few_shot_examples + df["Sentence"][i] + " " + str(df['Sentiment'][i]) + "\n"
print(few_shot_examples)

Based upon its unique fiber expertise and innovative approach , the company has a strong market position in several business areas in which it operates . 2
Finnish software developer Done Solutions Oyj said its net profit increased to 3.5 mln euro ( $ 4.6 mln ) in 2006 from 2.3 mln euro ( $ 3.0 mln ) in 2005 . 2
$ETN UPGRADE today by MS to overweight.  Excellent company and leadership 2



In [7]:
#test for 3-shots evaluation    
openai.api_type = "azure" 
openai.api_base = "https://bsaiuks.openai.azure.com/"
openai.api_version = "2023-03-15-preview" 
openai.api_key = "25d9022d3d124b9ba44ad4464ce14b3a"

sentence_input = full_df["text"][27]
response = openai.ChatCompletion.create( 
engine="gpt-4", 
messages = [{"role":"system","content":"You will be provided with a sentence, \
                                                and your task is to classify its sentiment is \
                                                hate or nothate. hate label as 1, nothate label as 0 \
                                                Answer:"},
                    {"role":"user","content":sentence_input},], 
                    temperature=0.7, 
                    max_tokens=800, 
                    top_p=0.95, 
                    frequency_penalty=0,
                    presence_penalty=0, 
                    stop=None
                    )
p = response["choices"][0]["message"].content
p

'0'